# Hyper-parameter Tunning

#### 1. Import Libraries

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
! pip install nni

  Using cached nni-2.4-py3-none-win_amd64.whl (53.4 MB)
  Using cached prettytable-2.2.1-py3-none-any.whl (23 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.9
    Uninstalling urllib3-1.25.9:
      Successfully uninstalled urllib3-1.25.9


ERROR: requests 2.24.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.7 which is incompatible.
ERROR: huggingface-hub 0.0.17 has requirement packaging>=20.9, but you'll have packaging 20.4 which is incompatible.


In [5]:
import tensorflow as tf
import nni

#### 2. Loading Dataset

In [6]:
def load_dataset():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    return (x_train/255., y_train), (x_test/255., y_test)

#### 3. Creating Model

In [7]:
def create_model(num_units, dropout_rate, lr, activation):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_units, activation=activation),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(10, activation="softmax")
    ])

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(lr=lr),
        metrics=["accuracy"]
    )
    return model

In [8]:
class ReportIntermediates(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        acc = logs.get("val_accuracy") or 0.
        nni.report_intermediate_result(acc)

In [9]:
def main(params):
    num_units = params.get("num_units")
    dropout_rate = params.get("dropout_rate")
    lr = params.get("lr")
    batch_size = params.get("batch_size")
    activation = params.get("activation")

    model = create_model(num_units, dropout_rate, lr, activation)

    (x_train, y_train), (x_test, y_test) = load_dataset()

    _ = model.fit(
        x_train, y_train,
        validation_data=(x_test, y_test),
        epochs=10,
        verbose=False,
        batch_size=batch_size,
        callbacks=[ReportIntermediates()]
    )

    _, acc = model.evaluate(x_test, y_test, verbose=False)
    nni.report_final_result(acc)

In [10]:
if __name__ == "__main__":
    params = {
        "num_units": 32,
        "dropout_rate": 0.1,
        "lr": 0.0001,
        "batch_size": 32,
        "activation": "relu"
    }

    tuned_params = nni.get_next_parameter()
    params.update(tuned_params)

    main(params)

[2021-09-29 19:19:39] INFO (nni/MainThread) Intermediate result: 0.8883000016212463  (Index 0)
[2021-09-29 19:19:42] INFO (nni/MainThread) Intermediate result: 0.9089999794960022  (Index 1)
[2021-09-29 19:19:45] INFO (nni/MainThread) Intermediate result: 0.9180999994277954  (Index 2)
[2021-09-29 19:19:47] INFO (nni/MainThread) Intermediate result: 0.9236999750137329  (Index 3)
[2021-09-29 19:19:50] INFO (nni/MainThread) Intermediate result: 0.9300000071525574  (Index 4)
[2021-09-29 19:19:53] INFO (nni/MainThread) Intermediate result: 0.9326000213623047  (Index 5)
[2021-09-29 19:19:56] INFO (nni/MainThread) Intermediate result: 0.9362000226974487  (Index 6)
[2021-09-29 19:19:59] INFO (nni/MainThread) Intermediate result: 0.939300000667572  (Index 7)
[2021-09-29 19:20:02] INFO (nni/MainThread) Intermediate result: 0.9412999749183655  (Index 8)
[2021-09-29 19:20:04] INFO (nni/MainThread) Intermediate result: 0.9440000057220459  (Index 9)
[2021-09-29 19:20:05] INFO (nni/MainThread) Final r